# Classification

## 1. Data Loading and Exploration

In [5]:
import pandas as pd
import numpy as np
import os

# File paths
data_dir = "../data"

# File names for training and test datasets
train_files = ["TrainData1.txt", "TrainData2.txt", "TrainData3.txt", "TrainData4.txt", "TrainData5.txt"]
train_label_files = ["TrainLabel1.txt", "TrainLabel2.txt", "TrainLabel3.txt", "TrainLabel4.txt", "TrainLabel5.txt"]
test_files = ["TestData1.txt", "TestData2.txt", "TestData3.txt", "TestData4.txt", "TestData5.txt"]

# Load each dataset into separate variables
train_datasets = {}
train_labels = {}
test_datasets = {}

for i in range(len(train_files)):
    # Load training data and label using read_table for whitespace-separated data
    train_datasets[f"train_data_{i+1}"] = pd.read_table(os.path.join(data_dir, train_files[i]), header=None, delim_whitespace=True)
    train_labels[f"train_label_{i+1}"] = pd.read_table(os.path.join(data_dir, train_label_files[i]), header=None, delim_whitespace=True)
    test_datasets[f"test_data_{i+1}"] = pd.read_table(os.path.join(data_dir, test_files[i]), header=None, delim_whitespace=True)
    
    # Display basic information for each dataset
    print(f"Train Data {i+1} - Dimensions: {train_datasets[f'train_data_{i+1}'].shape}")
    print(f"Train Label {i+1} - Dimensions: {train_labels[f'train_label_{i+1}'].shape}")
    print(f"Test Data {i+1} - Dimensions: {test_datasets[f'test_data_{i+1}'].shape}")
    
    # Check for missing values
    missing_count = (train_datasets[f"train_data_{i+1}"] == 1.00000000000000e+99).sum().sum()
    print(f"Train Data {i+1} - Missing Values: {missing_count}")
    
    missing_count_test = (test_datasets[f"test_data_{i+1}"] == 1.00000000000000e+99).sum().sum()
    print(f"Test Data {i+1} - Missing Values: {missing_count_test}")
    print("-" * 40)


Train Data 1 - Dimensions: (150, 3312)
Train Label 1 - Dimensions: (150, 1)
Test Data 1 - Dimensions: (53, 3312)
Train Data 1 - Missing Values: 9936
Test Data 1 - Missing Values: 7021
----------------------------------------
Train Data 2 - Dimensions: (100, 9182)
Train Label 2 - Dimensions: (100, 1)
Test Data 2 - Dimensions: (74, 9182)
Train Data 2 - Missing Values: 0
Test Data 2 - Missing Values: 0
----------------------------------------
Train Data 3 - Dimensions: (6300, 13)
Train Label 3 - Dimensions: (6300, 1)
Test Data 3 - Dimensions: (2693, 13)
Train Data 3 - Missing Values: 1886
Test Data 3 - Missing Values: 0
----------------------------------------
Train Data 4 - Dimensions: (2547, 112)
Train Label 4 - Dimensions: (2547, 1)
Test Data 4 - Dimensions: (1092, 112)
Train Data 4 - Missing Values: 0
Test Data 4 - Missing Values: 0
----------------------------------------
Train Data 5 - Dimensions: (1119, 11)
Train Label 5 - Dimensions: (1119, 1)
Test Data 5 - Dimensions: (480, 11)
T